In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm


In [14]:
video_games_df = pd.read_csv('video_games_df_final.csv')

# Post Matching Functions

In [3]:
def cate(matches, months):
    cate_dict = {month: {m: [] for m in months[s + 1:]} for s, month in enumerate(months[:-1])}
    
    for start, month in enumerate(months[:-1]):
        for other_month in months[start + 1:]:
            relevant_matches = matches[month][other_month]
            unit_sales = relevant_matches['current_sales'][0]
            matched_sales = relevant_matches['matched_sales'][0]
            mean_y = np.mean(matched_sales)
            cate_dict[month][other_month] = unit_sales - mean_y
            
    return cate_dict


def print_ates(cate_dict):
    for month, other_months in cate_dict.items():
        for other_month, cate_list in other_months.items():
            ate = np.mean(cate_list)
            print(f"ATE from {month} to {other_month}: {ate}")
            

# General Matching

In [4]:
T = video_games_df['month']
y = video_games_df['total_sales']
X = video_games_df.drop(columns=['month', 'total_sales', 'img', 'na_sales', 'jp_sales', 'pal_sales',
       'other_sales', 'release_date', 'last_update', 'iso_year',
       'iso_week', 'year_week', 'title'])

In [5]:
def find_closest_matches(video_games_df, k=1):
    # Initialize an empty dictionary to store the closest matches for each month
    closest_matches = {}
    
    # List of all unique months in the data
    months = video_games_df['month'].unique()

    # Loop over each month to find closest matches from other months
    for month in months:
        # Filter data for the current month and the rest of the months
        current_month_data = video_games_df[video_games_df['month'] == month]
        other_months_data = video_games_df[video_games_df['month'] != month]
        
        # Define feature sets for KNN
        X_current = current_month_data.drop(columns=['month', 'total_sales', 'img', 'na_sales', 
                                                     'jp_sales', 'pal_sales', 'other_sales', 
                                                     'release_date', 'last_update', 'iso_year', 
                                                     'iso_week', 'year_week', 'title'])
        X_other = other_months_data.drop(columns=['month', 'total_sales', 'img', 'na_sales', 
                                                  'jp_sales', 'pal_sales', 'other_sales', 
                                                  'release_date', 'last_update', 'iso_year', 
                                                  'iso_week', 'year_week', 'title'])
        
        # Initialize and fit the KNN model
        knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
        knn.fit(X_other)
        
        # Find k closest matches for each entry in the current month
        distances, indices = knn.kneighbors(X_current)
        
        # Store the results in the dictionary
        closest_matches[month] = {
            "distances": distances,
            "indices": indices,
            "matches": other_months_data.iloc[indices.flatten()]  # Closest matches data
        }
    
    return closest_matches


In [6]:
def find_closest_matches_with_sales(video_games_df, k=1):
    # Initialize a dictionary to store the closest matches for each unit by month
    closest_matches = {}

    # List of all unique months in the data
    months = video_games_df['month'].unique()

    # Loop over each month to get the current month's units
    for current_month in months:
        current_month_data = video_games_df[video_games_df['month'] == current_month]
        
        # Drop non-feature columns from current month's data
        X_current = current_month_data.drop(columns=['month', 'total_sales', 'img', 'na_sales', 
                                                     'jp_sales', 'pal_sales', 'other_sales', 
                                                     'release_date', 'last_update', 'iso_year', 
                                                     'iso_week', 'year_week', 'title'])
        
        # Initialize a dictionary to hold matches for each unit in the current month
        closest_matches[current_month] = {}

        # Loop over each of the other months
        for other_month in tqdm(months):
            if other_month == current_month:
                continue  # Skip the current month itself
            
            # Get data from the other month
            other_month_data = video_games_df[video_games_df['month'] == other_month]
            X_other = other_month_data.drop(columns=['month', 'total_sales', 'img', 'na_sales', 
                                                     'jp_sales', 'pal_sales', 'other_sales', 
                                                     'release_date', 'last_update', 'iso_year', 
                                                     'iso_week', 'year_week', 'title'])
            
            # Initialize and fit the KNN model on the other month's data
            knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
            knn.fit(X_other)
            
            # Find k closest matches for each unit in the current month from the other month
            distances, indices = knn.kneighbors(X_current)
            
            # Store the results in the dictionary under the current and other month combination
            closest_matches[current_month][other_month] = {
                "distances": distances,
                "indices": indices,
                "matches": other_month_data.iloc[indices.flatten()],
                "current_sales": current_month_data['total_sales'].values,  # Sales of current month units
                "matched_sales": other_month_data['total_sales'].iloc[indices.flatten()].values  # Sales of matched units
            }
    
    return closest_matches


## One neighbor

In [29]:
closest_matches = find_closest_matches_with_sales(video_games_df, k=1)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:23<00:00,  1.93s/it]


In [30]:
cate_dict = cate(closest_matches, video_games_df['month'].unique())
print_ates(cate_dict)

ATE from 9 to 11: 20.31
ATE from 9 to 10: 20.27
ATE from 9 to 4: 19.76
ATE from 9 to 6: 20.14
ATE from 9 to 5: 20.31
ATE from 9 to 8: 20.29
ATE from 9 to 3: 20.23
ATE from 9 to 1: 19.75
ATE from 9 to 7: 19.87
ATE from 9 to 2: 20.32
ATE from 9 to 12: 20.22
ATE from 11 to 10: 19.38
ATE from 11 to 4: 19.26
ATE from 11 to 6: 19.39
ATE from 11 to 5: 19.39
ATE from 11 to 8: 18.57
ATE from 11 to 3: 19.240000000000002
ATE from 11 to 1: 19.36
ATE from 11 to 7: 19.38
ATE from 11 to 2: 19.37
ATE from 11 to 12: 19.32
ATE from 10 to 4: 15.87
ATE from 10 to 6: 16.04
ATE from 10 to 5: 15.999999999999998
ATE from 10 to 8: 15.889999999999999
ATE from 10 to 3: 15.729999999999999
ATE from 10 to 1: 15.959999999999999
ATE from 10 to 7: 16.09
ATE from 10 to 2: 16.15
ATE from 10 to 12: 15.999999999999998
ATE from 4 to 6: 11.07
ATE from 4 to 5: 11.0
ATE from 4 to 8: 11.08
ATE from 4 to 3: 11.03
ATE from 4 to 1: 11.07
ATE from 4 to 7: 11.09
ATE from 4 to 2: 11.09
ATE from 4 to 12: 11.0
ATE from 6 to 5: 7.8
ATE

## Three Neighbors

In [31]:
closest_matches = find_closest_matches_with_sales(video_games_df, k=3)

100%|██████████| 12/12 [00:10<00:00,  1.13it/s]


In [32]:
cate_dict = cate(closest_matches, video_games_df['month'].unique())
print_ates(cate_dict)

ATE from 9 to 11: 20.31
ATE from 9 to 10: 20.27
ATE from 9 to 4: 19.76
ATE from 9 to 6: 20.14
ATE from 9 to 5: 20.31
ATE from 9 to 8: 20.29
ATE from 9 to 3: 20.23
ATE from 9 to 1: 19.75
ATE from 9 to 7: 19.87
ATE from 9 to 2: 20.32
ATE from 9 to 12: 20.22
ATE from 11 to 10: 19.38
ATE from 11 to 4: 19.26
ATE from 11 to 6: 19.39
ATE from 11 to 5: 19.39
ATE from 11 to 8: 18.57
ATE from 11 to 3: 19.240000000000002
ATE from 11 to 1: 19.36
ATE from 11 to 7: 19.38
ATE from 11 to 2: 19.37
ATE from 11 to 12: 19.32
ATE from 10 to 4: 15.87
ATE from 10 to 6: 16.04
ATE from 10 to 5: 15.999999999999998
ATE from 10 to 8: 15.889999999999999
ATE from 10 to 3: 15.729999999999999
ATE from 10 to 1: 15.959999999999999
ATE from 10 to 7: 16.09
ATE from 10 to 2: 16.15
ATE from 10 to 12: 15.999999999999998
ATE from 4 to 6: 11.07
ATE from 4 to 5: 11.0
ATE from 4 to 8: 11.08
ATE from 4 to 3: 11.03
ATE from 4 to 1: 11.07
ATE from 4 to 7: 11.09
ATE from 4 to 2: 11.09
ATE from 4 to 12: 11.0
ATE from 6 to 5: 7.8
ATE

## Five Neigbors

In [33]:
closest_matches = find_closest_matches_with_sales(video_games_df, k=5)

100%|██████████| 12/12 [00:10<00:00,  1.12it/s]


In [34]:
cate_dict = cate(closest_matches, video_games_df['month'].unique())
print_ates(cate_dict)

ATE from 9 to 11: 20.31
ATE from 9 to 10: 20.27
ATE from 9 to 4: 19.76
ATE from 9 to 6: 20.14
ATE from 9 to 5: 20.31
ATE from 9 to 8: 20.29
ATE from 9 to 3: 20.23
ATE from 9 to 1: 19.75
ATE from 9 to 7: 19.87
ATE from 9 to 2: 20.32
ATE from 9 to 12: 20.22
ATE from 11 to 10: 19.38
ATE from 11 to 4: 19.26
ATE from 11 to 6: 19.39
ATE from 11 to 5: 19.39
ATE from 11 to 8: 18.57
ATE from 11 to 3: 19.240000000000002
ATE from 11 to 1: 19.36
ATE from 11 to 7: 19.38
ATE from 11 to 2: 19.37
ATE from 11 to 12: 19.32
ATE from 10 to 4: 15.87
ATE from 10 to 6: 16.04
ATE from 10 to 5: 15.999999999999998
ATE from 10 to 8: 15.889999999999999
ATE from 10 to 3: 15.729999999999999
ATE from 10 to 1: 15.959999999999999
ATE from 10 to 7: 16.09
ATE from 10 to 2: 16.15
ATE from 10 to 12: 15.999999999999998
ATE from 4 to 6: 11.07
ATE from 4 to 5: 11.0
ATE from 4 to 8: 11.08
ATE from 4 to 3: 11.03
ATE from 4 to 1: 11.07
ATE from 4 to 7: 11.09
ATE from 4 to 2: 11.09
ATE from 4 to 12: 11.0
ATE from 6 to 5: 7.8
ATE

# Propensity Scores

In [8]:
import numpy as np

In [10]:
# 1. Preprocess the data
# Impute missing values for 'critic_score'
imputer = SimpleImputer(strategy='mean')
video_games_df['critic_score'] = imputer.fit_transform(video_games_df[['critic_score']])

# Convert categorical variables to one-hot encoding
cat_columns = ['console', 'genre', 'publisher', 'developer']
video_games_df = pd.get_dummies(video_games_df, columns=cat_columns, drop_first=True)

# Separate features and target
X = video_games_df.drop(columns=['month', 'total_sales', 'img', 'na_sales', 'jp_sales', 'pal_sales',
       'other_sales', 'release_date', 'last_update', 'iso_year',
       'iso_week', 'year_week', 'title'])
y = video_games_df['month']

# 2. Estimate Propensity Scores
# Fit a multinomial logistic regression for propensity scores
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
logistic_model.fit(X, y)
propensity_scores = logistic_model.predict_proba(X)  # Each row has probabilities for each 'month'

# 3. Match Units
# Compute pairwise distances based on propensity scores
# This creates a distance matrix for matching similar propensity scores across months
distances = pairwise_distances(propensity_scores, metric='euclidean')


C:\Users\moriy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Save propensity scores
np.save("propensity_scores.npy", propensity_scores)  # Binary format
np.savetxt("propensity_scores.csv", propensity_scores, delimiter=",")  # Text format (optional)

# Save distances
np.save("distances.npy", distances)  # Binary format
np.savetxt("distances.csv", distances, delimiter=",")  # Text format (optional)


In [9]:
propensity_scores = np.load("propensity_scores.npy")

In [10]:
def find_closest_matches_with_sales_and_propensity(video_games_df, propensity_scores, k=1):
    # Initialize a dictionary to store the closest matches for each unit by month
    closest_matches = {}

    # List of all unique months in the data
    months = video_games_df['month'].unique()

    # Loop over each month to get the current month's units
    for current_month in months:
        current_month_data = video_games_df[video_games_df['month'] == current_month]
        
        # Drop non-feature columns from current month's data
        X_current = current_month_data.drop(columns=['month', 'total_sales', 'img', 'na_sales', 
                                                     'jp_sales', 'pal_sales', 'other_sales', 
                                                     'release_date', 'last_update', 'iso_year', 
                                                     'iso_week', 'year_week', 'title'])
        
        # Add the propensity scores corresponding to the current month
        current_month_indices = current_month_data.index
        propensity_current = propensity_scores[current_month_indices]
        X_current_combined = np.hstack((X_current, propensity_current))
        
        # Initialize a dictionary to hold matches for each unit in the current month
        closest_matches[current_month] = {}

        # Loop over each of the other months
        for other_month in tqdm(months):
            if other_month == current_month:
                continue  # Skip the current month itself
            
            # Get data from the other month
            other_month_data = video_games_df[video_games_df['month'] == other_month]
            X_other = other_month_data.drop(columns=['month', 'total_sales', 'img', 'na_sales', 
                                                     'jp_sales', 'pal_sales', 'other_sales', 
                                                     'release_date', 'last_update', 'iso_year', 
                                                     'iso_week', 'year_week', 'title'])
            
            # Add the propensity scores corresponding to the other month
            other_month_indices = other_month_data.index
            propensity_other = propensity_scores[other_month_indices]
            X_other_combined = np.hstack((X_other, propensity_other))

            # Initialize and fit the KNN model on the other month's data
            knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
            knn.fit(X_other_combined)
            
            # Find k closest matches for each unit in the current month from the other month
            distances, indices = knn.kneighbors(X_current_combined)
            
            # Store the results in the dictionary under the current and other month combination
            closest_matches[current_month][other_month] = {
                "distances": distances,
                "indices": indices,
                "matches": other_month_data.iloc[indices.flatten()],
                "current_sales": current_month_data['total_sales'].values,  # Sales of current month units
                "matched_sales": other_month_data['total_sales'].iloc[indices.flatten()].values  # Sales of matched units
            }
    
    return closest_matches

In [15]:
imputer = SimpleImputer(strategy='mean')
video_games_df['critic_score'] = imputer.fit_transform(video_games_df[['critic_score']])

# Convert categorical variables to one-hot encoding
cat_columns = ['console', 'genre', 'publisher', 'developer']
video_games_df = pd.get_dummies(video_games_df, columns=cat_columns, drop_first=True)
matches = find_closest_matches_with_sales_and_propensity(video_games_df, propensity_scores)

100%|██████████| 12/12 [00:32<00:00,  2.72s/it]


In [16]:
cate_dict = cate(matches, video_games_df['month'].unique())
print_ates(cate_dict)

ATE from 9 to 11: 20.31
ATE from 9 to 10: 20.27
ATE from 9 to 4: 19.76
ATE from 9 to 6: 20.14
ATE from 9 to 5: 20.31
ATE from 9 to 8: 20.29
ATE from 9 to 3: 20.23
ATE from 9 to 1: 19.75
ATE from 9 to 7: 19.87
ATE from 9 to 2: 20.32
ATE from 9 to 12: 20.22
ATE from 11 to 10: 19.38
ATE from 11 to 4: 19.26
ATE from 11 to 6: 19.39
ATE from 11 to 5: 19.39
ATE from 11 to 8: 18.57
ATE from 11 to 3: 19.240000000000002
ATE from 11 to 1: 19.36
ATE from 11 to 7: 19.38
ATE from 11 to 2: 19.37
ATE from 11 to 12: 19.32
ATE from 10 to 4: 15.87
ATE from 10 to 6: 16.04
ATE from 10 to 5: 15.999999999999998
ATE from 10 to 8: 15.889999999999999
ATE from 10 to 3: 15.729999999999999
ATE from 10 to 1: 15.959999999999999
ATE from 10 to 7: 16.09
ATE from 10 to 2: 16.15
ATE from 10 to 12: 15.999999999999998
ATE from 4 to 6: 11.07
ATE from 4 to 5: 11.0
ATE from 4 to 8: 11.08
ATE from 4 to 3: 11.03
ATE from 4 to 1: 11.07
ATE from 4 to 7: 11.09
ATE from 4 to 2: 11.09
ATE from 4 to 12: 11.0
ATE from 6 to 5: 7.8
ATE